In [3]:
!pip install fastapi

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)


In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pymysql


  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
from fastapi import APIRouter, HTTPException, Query
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from typing import List, Optional, Dict, Any
from datetime import datetime
import traceback
from sqlalchemy import text

# 既存のモジュールをインポート
from db_control import crud
from db_control.connect_MySQL import engine
from db_control.mymodels_MySQL import User, UserTag, Tag, PointTransaction, Event, EventTag, Store
from db_control.crud import session_scope

In [30]:
# レスポンスモデルの定義
class EventRecommendation(BaseModel):
    id: str
    imageUrl: Optional[str] = None
    area: Optional[str] = None
    title: str
    date: Optional[str] = None
    tags: List[str] = []
    description: Optional[str] = None
    points: Optional[int] = None

In [46]:
class RecommendationResponse(BaseModel):
    events: List[EventRecommendation]
    similarUsers: List[int] = []

In [32]:
# データ取得関数
def get_user_data(session) -> pd.DataFrame:
    """ユーザーデータを取得し前処理する"""
    query_users = text("""
    SELECT user_id, gender, relationship_id, postal_code, birth_date 
    FROM Users
    """)
    result_users = session.execute(query_users).fetchall()
    if not result_users:
        raise HTTPException(status_code=404, detail="ユーザーデータが見つかりません")
    
    # 結果をDataFrameに変換
    df_users = pd.DataFrame(result_users)
    
    # 性別を数値に変換
    df_users["gender"] = df_users["gender"].map({"M": 0, "F": 1, "U": 2})
    
    # 年齢の計算
    df_users["age"] = pd.to_datetime("today").year - pd.to_datetime(df_users["birth_date"]).dt.year
    df_users.drop(columns=["birth_date"], inplace=True)
    
    # 年齢のMin-Maxスケーリング
    scaler = MinMaxScaler()
    df_users["age"] = scaler.fit_transform(df_users[["age"]])
    
    # 郵便番号をワンホットエンコーディング
    df_users = pd.get_dummies(df_users, columns=["postal_code"])
    return df_users

実行テスト

In [33]:
with session_scope() as session:
    df_users = get_user_data(session)

# データ確認
print(df_users.shape)  # 行数と列数
print(df_users.columns)  # カラム名一覧
df_users.head()  # 先頭5件

2025-04-20 15:47:46,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-20 15:47:46,105 INFO sqlalchemy.engine.Engine 
    SELECT user_id, gender, relationship_id, postal_code, birth_date 
    FROM Users
    
2025-04-20 15:47:46,107 INFO sqlalchemy.engine.Engine [cached since 1.4e+05s ago] {}
2025-04-20 15:47:46,301 INFO sqlalchemy.engine.Engine COMMIT
(31, 5)
Index(['user_id', 'gender', 'relationship_id', 'age', 'postal_code_8112419'], dtype='object')


,user_id,gender,relationship_id,age,postal_code_8112419
0,1,0,1.0,0.890625,False
1,2,0,1.0,0.953125,False
2,3,0,1.0,1.000000,False
3,4,1,2.0,0.859375,False
4,5,1,2.0,0.984375,False


In [34]:
def get_user_tags(session) -> pd.DataFrame:
    """ユーザータグデータを取得しワンホットエンコーディングする"""
    query_tags = text("""
    SELECT u.user_id, t.tag_name  
    FROM UserTags u 
    JOIN Tags t ON u.tag_id = t.tag_id
    """)
    result_tags = session.execute(query_tags).fetchall()
    
    # タグのワンホットエンコーディング
    if result_tags:
        df_tags = pd.DataFrame(result_tags)
        df_tags_onehot = df_tags.pivot_table(
            index="user_id", columns="tag_name", aggfunc="size", fill_value=0
        )
        return df_tags_onehot.add_prefix("tag_")
    else:
        # タグがない場合は空のDataFrameを作成
        return pd.DataFrame()

In [35]:
with session_scope() as session:
    df_tags_onehot = get_user_tags(session)

# データ確認
print(df_tags_onehot.shape)  # 行数と列数
print(df_tags_onehot.columns)  # カラム名一覧
df_tags_onehot.head()  # 先頭5件

2025-04-20 15:48:28,671 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-20 15:48:28,679 INFO sqlalchemy.engine.Engine 
    SELECT u.user_id, t.tag_name  
    FROM UserTags u 
    JOIN Tags t ON u.tag_id = t.tag_id
    
2025-04-20 15:48:28,680 INFO sqlalchemy.engine.Engine [cached since 1.393e+05s ago] {}
2025-04-20 15:48:29,043 INFO sqlalchemy.engine.Engine COMMIT
(31, 35)
Index(['tag_active', 'tag_art', 'tag_community', 'tag_contribute',
       'tag_digital', 'tag_experience', 'tag_sports', 'tag_temple',
       'tag_volunteer', 'tag_お祭り', 'tag_のんびり派', 'tag_アウトドア', 'tag_アクティブ',
       'tag_アート', 'tag_ウォーキング', 'tag_エコ志向', 'tag_エンタメ', 'tag_カフェ巡り', 'tag_グルメ',
       'tag_スイーツ', 'tag_スポーツ', 'tag_デジタル', 'tag_ボランティア', 'tag_ライフスタイル',
       'tag_ライブイベント', 'tag_伝統工芸', 'tag_健康志向', 'tag_地元グルメ', 'tag_地域活性化',
       'tag_子育て支援', 'tag_学び・体験', 'tag_文化・歴史', 'tag_社会貢献', 'tag_神社仏閣',
       'tag_食べ歩き'],
      dtype='object', name='tag_name')


tag_name,tag_active,tag_art,tag_community,tag_contribute,tag_digital,tag_experience,tag_sports,tag_temple,tag_volunteer,tag_お祭り,...,tag_伝統工芸,tag_健康志向,tag_地元グルメ,tag_地域活性化,tag_子育て支援,tag_学び・体験,tag_文化・歴史,tag_社会貢献,tag_神社仏閣,tag_食べ歩き
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
def get_transaction_data(session) -> pd.DataFrame:
    """ポイント取引データを取得しワンホットエンコーディングする"""
    query_transactions = text("""
    SELECT user_id, store_id 
    FROM PointTransaction 
    WHERE user_id IS NOT NULL
    """)
    result_transactions = session.execute(query_transactions).fetchall()
    
    # 取引のワンホットエンコーディング
    if result_transactions:
        df_transactions = pd.DataFrame(result_transactions)
        df_transactions_onehot = df_transactions.pivot_table(
            index="user_id", columns="store_id", aggfunc="size", fill_value=0
        )
        return df_transactions_onehot.add_prefix("store_")
    else:
        # 取引がない場合は空のDataFrameを作成
        return pd.DataFrame()


In [37]:
with session_scope() as session:
    df_transactions_onehot = get_transaction_data(session)

# データ確認
print(df_transactions_onehot.shape)  # 行数と列数
print(df_transactions_onehot.columns)  # カラム名一覧
df_transactions_onehot.head()  # 先頭5件

2025-04-20 15:49:11,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-20 15:49:11,430 INFO sqlalchemy.engine.Engine 
    SELECT user_id, store_id 
    FROM PointTransaction 
    WHERE user_id IS NOT NULL
    
2025-04-20 15:49:11,430 INFO sqlalchemy.engine.Engine [cached since 1.392e+05s ago] {}
2025-04-20 15:49:11,531 INFO sqlalchemy.engine.Engine COMMIT
(22, 6)
Index(['store_1', 'store_2', 'store_3', 'store_4', 'store_5', 'store_6'], dtype='object', name='store_id')


store_id,store_1,store_2,store_3,store_4,store_5,store_6
user_id,,,,,,
1,5,0,0,0,0,0
2,0,3,0,0,0,0
3,3,0,3,0,1,0
4,1,0,0,3,1,0
5,0,0,0,1,3,1


In [42]:
def get_favorite_events_onehot(session) -> pd.DataFrame:
    """お気に入りイベントをワンホットエンコーディングする"""
    query_favorites = text("""
    SELECT user_id, event_id
    FROM FavoriteEvents
    """)
    result_fav = session.execute(query_favorites).fetchall()
    
    if result_fav:
        df_fav = pd.DataFrame(result_fav)
        df_fav_onehot = df_fav.pivot_table(
            index="user_id", columns="event_id", aggfunc="size", fill_value=0
        )
        return df_fav_onehot.add_prefix("fav_event_")
    else:
        return pd.DataFrame()


In [41]:
def get_favorite_events_onehot(session) -> pd.DataFrame:
    """お気に入りイベントをワンホットエンコーディングする"""
    query_fav = text("""
    SELECT user_id, event_id
    FROM FavoriteEvents
    """)
    result_fav = session.execute(query_fav).fetchall()

    if result_fav:
        df_fav = pd.DataFrame(result_fav)
        df_fav_onehot = df_fav.pivot_table(
            index="user_id", columns="event_id", aggfunc="size", fill_value=0
        )
        return df_fav_onehot.add_prefix("fav_event_")
    else:
        return pd.DataFrame(index=df_users["user_id"].unique())  # ←ここ注意


In [43]:
with session_scope() as session:
    df_fav_onehot = get_favorite_events_onehot(session)

# データ確認
print(df_fav_onehot.shape)  # 行数と列数
print(df_fav_onehot.columns)  # カラム名一覧
df_fav_onehot.head()  # 先頭5件

2025-04-20 16:10:35,691 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-20 16:10:35,692 INFO sqlalchemy.engine.Engine 
    SELECT user_id, event_id
    FROM FavoriteEvents
    
2025-04-20 16:10:35,692 INFO sqlalchemy.engine.Engine [cached since 1.401e+05s ago] {}
2025-04-20 16:10:35,782 INFO sqlalchemy.engine.Engine COMMIT
(1, 2)
Index(['fav_event_75', 'fav_event_96'], dtype='object', name='event_id')


event_id,fav_event_75,fav_event_96
user_id,,
3,1,1


In [44]:
def calculate_recommendations(user_id: int, top_n: int = 5) -> Dict[str, Any]:
    """協調フィルタリングによるイベント推薦を計算する"""
    try:
        with session_scope() as session:
            # 1. 各種データの取得
            df_users = get_user_data(session)
            df_tags_onehot = get_user_tags(session)
            df_transactions_onehot = get_transaction_data(session)
            df_fav_events_onehot = get_favorite_events_onehot(session)
                        
            # 2. データの統合
            area_columns = [col for col in df_users.columns if col.startswith("postal_code")]
            df_users[area_columns] = df_users[area_columns].astype(int)
            
            # データフレームを結合
            df_final = df_users.set_index("user_id")
            
            # タグデータがあれば結合
            if not df_tags_onehot.empty:
                df_final = df_final.join(df_tags_onehot, how="left")
            
            # 取引データがあれば結合
            if not df_transactions_onehot.empty:
                df_final = df_final.join(df_transactions_onehot, how="left")

            if not df_fav_events_onehot.empty:
                df_final = df_final.join(df_fav_events_onehot, how="left")
            
            # 欠損値を0で埋める
            df_final = df_final.fillna(0)
            
            # 3. 類似ユーザーの特定
            similar_users = find_similar_users(df_final, user_id, top_n)
            if not similar_users:
                return {"events": [], "similarUsers": []}
            
            # # 4. 類似ユーザーが訪れた店舗の特定
            # store_ids = find_recommended_stores(df_transactions_onehot, similar_users)
            # if not store_ids:
            #     return {"events": [], "similarUsers": similar_users}
            
            # # 5. 店舗に関連するイベントの取得
            # events_data = get_events_by_store_ids(session, store_ids)
           
            # # 6. イベント情報をフォーマット
            # recommended_events = [
            #     format_event_to_recommendation(session, event) 
            #     for event in events_data
            # ]

            # 4. 類似ユーザーがお気に入り登録しているイベントを取得
            query = text("""
            SELECT DISTINCT e.event_id, e.event_name, e.description, e.start_date, e.end_date,
                            e.flyer_url, e.event_image_url, e.store_id
            FROM FavoriteEvents f
            JOIN Events e ON f.event_id = e.event_id
            WHERE f.user_id IN :similar_users
            ORDER BY e.start_date ASC
            LIMIT 6
            """)

            # SQLAlchemyにリストを渡す場合はタプル形式にする必要あり
            result_events = session.execute(query, {"similar_users": tuple(similar_users)}).fetchall()

            # 5. イベント情報をフォーマット
            recommended_events = [
                format_event_to_recommendation(session, event)
                for event in result_events
            ]
            
            return {
                "events": recommended_events,
                "similarUsers": similar_users
            }
        print(similar_users)
            
    except Exception as e:
        print(f"推薦計算エラー: {str(e)}")
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"推薦計算中にエラーが発生しました: {str(e)}")

In [45]:
print(similar_users)

NameError: name 'similar_users' is not defined